In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
from encodings import euc_kr
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

In [3]:
f=pd.read_csv('final_merge.csv', encoding='euc_kr')

In [8]:
f[f['날짜']=='2016-12-29']

,날짜,지수종가,거래대금,상장주식수,시가총액,외국인보유주식수,외국인보유시가총액,신용거래종목수,신용자료일자,회사수,...,금속,oil,미국n년,국고채권(n년),USD,EUR,CNY,JPY,GBP,Y
3961,2016-12-29,2026.46,3009199,41031721,1308440373,6648261,460632479,1282.0,20161227.0,779,...,10995.0,55.0,2.5,3.1,1207.6,1258.44,173.63,1031.7,1476.11,1


In [5]:
for i in range(f.shape[0]):
    if f.iloc[i,-1]=='상승':
        f.iloc[i,-1]=1
    else:
        f.iloc[i,-1]=0


In [9]:
X_train=f.iloc[0:3962,1:-1]
X_test=f.iloc[3962:,1:-1]
y_train=f.iloc[0:3962,-1]
y_test=f.iloc[3962:,-1]

In [10]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

X_train=np.where(np.isnan(X_train), 0, X_train)
X_test=np.where(np.isnan(X_test), 0, X_test)

In [11]:
from sklearn.decomposition import PCA
pca=PCA(n_components=11)

X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)
print('PCA component 별 변동성:', pca.explained_variance_ratio_)

PCA component 별 변동성: [0.39464929 0.12131351 0.09002742 0.07924873 0.05469566 0.04234121
 0.03539075 0.02893365 0.02385328 0.02165712 0.01718348]


In [12]:
log_clf=LogisticRegression(random_state=42)
rf_clf=RandomForestClassifier(random_state=42)
svm_clf=SVC(random_state=42)
nn_clf=KNeighborsClassifier(n_neighbors=5)

voting_clf=VotingClassifier(estimators=[('lr', log_clf), ('rf', rf_clf), ('svc',svm_clf),('nn', nn_clf)], voting='hard')
voting_clf.fit(X_train, y_train)

C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                             

In [13]:
for clf in (log_clf, rf_clf, svm_clf, nn_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))
    print(clf.__class__.__name__,confusion_matrix(y_test,y_pred))
    print(clf.__class__.__name__,f1_score(y_test,y_pred))

C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


LogisticRegression 0.5544147843942505
LogisticRegression [[  1 217]
 [  0 269]]
LogisticRegression 0.7125827814569535
RandomForestClassifier 0.5379876796714579
RandomForestClassifier [[ 95 123]
 [102 167]]
RandomForestClassifier 0.5974955277280859


C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC 0.5523613963039015
SVC [[  0 218]
 [  0 269]]
SVC 0.7116402116402116
KNeighborsClassifier 0.5462012320328542
KNeighborsClassifier [[ 27 191]
 [ 30 239]]
KNeighborsClassifier 0.6838340486409156


C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


VotingClassifier 0.5605749486652978
VotingClassifier [[ 21 197]
 [ 17 252]]
VotingClassifier 0.7019498607242339
